# Create Structures for Planets

This creates latex figures that act as tilesheets for each of the planets.

First we load the planet textures from the source code. Each character is stored from top left to bottom right.

In [121]:
import re
colors = {
    "00": "",
    "01": "\\BACKGROUNDONE",
    "10": "\\BACKGROUNDTWO",
    "11": "\\CHARCOLOR",
}
charsets_files = ["../iridisalpha/src/graphics/planet_textures.asm"]
charsets = {}
for charsets_file in charsets_files:
    input_file = open(charsets_file,'r')
    charset_data = []
    curr_charset = []
    for l in input_file.readlines():
        if "CHARACTER" in l or "planet" in l:
            if charset_data:
                curr_charset += [charset_data]
            charset_data = []
        if "planet" in l:
            if curr_charset:
                charsets[charset_name] = curr_charset
            charset_name = l.strip()
            curr_charset = []
            continue

        m = re.findall(r"[0-1]{8}",l)
        if not m:
            continue
        bits = m[0]
        for i in range(0,7,2):
            bitpair = bits[i:i+2]
            charset_data += [colors[bitpair]]
            charset_data += [colors[bitpair]]
    if charset_data:
        curr_charset += [charset_data]
    charsets[charset_name] = curr_charset


In [122]:
planet_colors = {
    'planet1Charset' : ['brown','lightblue','lightgreen'],
    'planet2Charset' : ['gray','lightred','brown'],
    'planet3Charset' : ['yellow','white', 'lightred'],
    'planet4Charset' : ['lightblue','yellow', 'darkgray'],
    'planet5Charset' : ['lightgreen','lightred','lightred'],
}

Get the structure data

In [123]:
raw_structure_data = """
mediumStructureData  .BYTE $65,$67,$69,$6B,$FF        
                     .BYTE $64,$66,$68,$6A,$FE        
largestStructureData .BYTE $41,$43,$51,$53,$41,$43,$FF
                     .BYTE $60,$60,$50,$52,$60,$60,$FF
                     .BYTE $49,$4B,$4D,$4F,$6D,$6F,$FF
                     .BYTE $48,$4A,$4C,$4E,$6C,$6E,$FE
nextLargestStructure .BYTE $59,$5B,$FF
                     .BYTE $58,$5A,$FF    
                     .BYTE $55,$57,$FF                
                     .BYTE $54,$56,$FE                
warpGateData         .BYTE $75,$77,$7D,$7F,$FF        
                     .BYTE $74,$76,$7C,$7E,$FF        
                     .BYTE $71,$73,$79,$7B,$FF        
                     .BYTE $70,$72,$78,$7A,$FE        
littleStructureData  .BYTE $45,$47,$FF
                     .BYTE $44,$46,$FE
"""
structure_data = {}
struct_chars = []
for l in raw_structure_data.split('\n'):
    if any(d in l for d in ["Data","Structure"]):
        if struct_chars:
            structure_data[struct_name] = struct_chars
        struct_name = l[:21].strip()
        struct_chars = [l[27:].strip().split(',')[:-1]]
        continue
    if "BYTE" in l:
        struct_chars += [l[27:].strip().split(',')[:-1]]
structure_data[struct_name] = struct_chars


Create `scenes`, a list containing the full scene to render in charset hex bytes. The scene runs from the top left to the bottom right.

In [133]:
print(structure_data.keys())

dict_keys(['mediumStructureData', 'largestStructureData', 'nextLargestStructure', 'warpGateData', 'littleStructureData'])


In [138]:
import copy
SCENE_WIDTH = 6
def createBlankScene():
    HEIGHT = 4
    blank_scene = []
    for y in range(0,HEIGHT):
        l = []
        for x in range(0,SCENE_WIDTH,2):
            l += ["$41","$43"] if y == 0 else ["$00","$00"]
        blank_scene += [l]
    return blank_scene

scenes = {}
offsets = {
    'mediumStructureData':  1, 
    'largestStructureData': 0,
    'nextLargestStructure' : 2, 
    'warpGateData': 1,
    'littleStructureData' : 2,
}
for struct_name, struct_data in structure_data.items():
    struct = createBlankScene()
    struct_offset = offsets[struct_name]
    for i,v in enumerate(struct_data):
        for j,b in enumerate(v):
            struct[i][struct_offset+j] = b
    scenes[struct_name] = list(reversed(copy.deepcopy(struct)))


Create the latex figure files in the `planets` subdirectory, with names like `charsetPlanet1_mediumStructureData`.

In [139]:
BIT_WIDTH = 8

def fillCharacter(c, byte_x_offset=0, byte_y_offset=0, grid_height=0):
    output = ""
    bit_y_offset = ((grid_height - (byte_y_offset * BIT_WIDTH))) - 1
    for j,v in enumerate(c):
        if not v:
            continue
        x = (byte_x_offset * BIT_WIDTH) + ((j % BIT_WIDTH) if j >= 7 else j)
        y = (bit_y_offset - int(j/BIT_WIDTH)) if j >= 7 else bit_y_offset
        output += f'\t\\fill[{v}] ({x},{y}) rectangle ++ (1,1);\n'
    return output

def createStructure(planet_name, scene_name, scene):
    grid_width = BIT_WIDTH * len(scene[0])
    grid_height = BIT_WIDTH * len(scene)

    header = """
\\begin{figure}[H]
  {
    \\setlength{\\tabcolsep}{3.0pt}
    \\setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
    \\begin{adjustbox}{width=3cm,center}
      \\begin{tikzpicture}
"""

    output = header + '\n'
    output += f"\\def\BACKGROUNDONE{{{planet_colors[planet_name][0]}}}\n"
    output += f"\\def\BACKGROUNDTWO{{{planet_colors[planet_name][1]}}}\n"
    output += f"\\def\CHARCOLOR{{{planet_colors[planet_name][2]}}}\n"

    #grid = f"\\draw[step=1.0,gray,very thin] (0,0) grid ({grid_width},{grid_height});"
    #output += f"\t{grid}\n"
    
    charset = charsets[planet_name]
    for byte_y_offset,ln in enumerate(scene):
        for byte_x_offset,hex_ref in enumerate(ln):
            char_index = int(hex_ref[1:],16) - 0x40
            if char_index < 0:
                continue
            c = charset[char_index]
            output += fillCharacter(c, byte_x_offset, byte_y_offset, grid_height)

    footer = f"""
      \\end{{tikzpicture}}
    \\end{{adjustbox}}
  }}\\caption{{{planet_name}\ {scene_name}}}
\\end{{figure}}
"""
    output += footer
    return output

for scene_name, scene in scenes.items():
    for planet_name in planet_colors:
        output = createStructure(planet_name, scene_name, scene)
        file_name = f"../src/planets/{planet_name}_{scene_name}.tex"
        output_file = open(file_name,'w')
        output_file.write(output)
        output_file.close()
        print(f"\subfile{{planets/{planet_name}_{scene_name}}}")


\subfile{planets/planet1Charset_mediumStructureData}
\subfile{planets/planet2Charset_mediumStructureData}
\subfile{planets/planet3Charset_mediumStructureData}
\subfile{planets/planet4Charset_mediumStructureData}
\subfile{planets/planet5Charset_mediumStructureData}
\subfile{planets/planet1Charset_largestStructureData}
\subfile{planets/planet2Charset_largestStructureData}
\subfile{planets/planet3Charset_largestStructureData}
\subfile{planets/planet4Charset_largestStructureData}
\subfile{planets/planet5Charset_largestStructureData}
\subfile{planets/planet1Charset_nextLargestStructure}
\subfile{planets/planet2Charset_nextLargestStructure}
\subfile{planets/planet3Charset_nextLargestStructure}
\subfile{planets/planet4Charset_nextLargestStructure}
\subfile{planets/planet5Charset_nextLargestStructure}
\subfile{planets/planet1Charset_warpGateData}
\subfile{planets/planet2Charset_warpGateData}
\subfile{planets/planet3Charset_warpGateData}
\subfile{planets/planet4Charset_warpGateData}
\subfile{pl